task name: Implement SdcaLogisticRegressionWithOptions
task description: Implement the provided C# code for SdcaLogisticRegressionWithOptions as a dotnet interactive notebook

In [ ]:
#r "nuget:Microsoft.ML"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
private class DataPoint
{
    public bool Label { get; set; }
    [VectorType(50)]
    public float[] Features { get; set; }
}

private static IEnumerable<DataPoint> GenerateRandomDataPoints(int count, int seed = 0)
{
    var random = new Random(seed);
    float randomFloat() => (float)random.NextDouble();
    for (int i = 0; i < count; i++)
    {
        var label = randomFloat() > 0.5f;
        yield return new DataPoint
        {
            Label = label,
            Features = Enumerable.Repeat(label, 50)
                .Select(x => x ? randomFloat() : randomFloat() + 0.03f).ToArray()
        };
    }
}

var dataPoints = GenerateRandomDataPoints(1000);
var trainingData = mlContext.Data.LoadFromEnumerable(dataPoints);
trainingData = mlContext.Data.Cache(trainingData);

In [ ]:
var options = new SdcaLogisticRegressionBinaryTrainer.Options()
{
    ConvergenceTolerance = 0.05f,
    MaximumNumberOfIterations = 30,
    PositiveInstanceWeight = 1.2f,
};
var pipeline = mlContext.BinaryClassification.Trainers.SdcaLogisticRegression(options);

In [ ]:
var model = pipeline.Fit(trainingData);

In [ ]:
var testData = mlContext.Data.LoadFromEnumerable(GenerateRandomDataPoints(500, seed: 123));
var transformedTestData = model.Transform(testData);

In [ ]:
private class Prediction
{
    // Original label.
    public bool Label { get; set; }
    // Predicted label from the trainer.
    public bool PredictedLabel { get; set; }
}

var predictions = mlContext.Data.CreateEnumerable<Prediction>(transformedTestData, reuseRowObject: false).ToList();
foreach (var p in predictions.Take(5))
    Console.WriteLine($"Label: {p.Label}, Prediction: {p.PredictedLabel}");